In [1]:
import pandas as pd
import torch
from torch.utils.data import Dataset

In [66]:
df = pd.read_csv('../input/jobs_input.csv' )#,names=["jobs"])

In [67]:
df = df[['jobs']]
# df = df['jobs']
df.head()

KeyError: "['jobs'] not in index"

In [59]:
df = df.sample(frac=1, random_state=120)

In [60]:
df

22                       عمال تشغيل اله طباعه ورق جدران
23                   عامل يشغل ماكينه تقطيع عجينه الورق
22                      عمال تشغيل اله طي وتغريه حراريه
24    عامل يشغل ماكينه صناعه منتجات الطحين مثل النشا...
24     تجهيز الخمور والمشروبات الروحيه الاخري صانع خمور
21                      عامل يجمع ويركب المعدات السمعيه
25                    عامل تشغيل ماكينه حياكه المفروشات
22                     عمال تشغيل اله تجهيز اطباق الورق
22                   عمال تشغيل اله طباعه نماذج الفلسكو
25                         عامل يشغل ماكينه حقائب السفر
20                       عامل فرمله في المناجم والمحاجر
25                       عامل تشغيل ماكينه تطريز صناعيه
21                      عامل يجمع ويركب ماكينات الطباعه
22                       عمال تشغيل اله قص شريطي (شيتر)
21                      عامل يجمع ويركب الالات الزراعيه
6                    عامل شفط المياه العادمه في الشوارع
22               وقاد مراجل بخاريه (عطشجي) محرك بالبخار
23                   عامل يشغل ماكينه تصفيه عجين

In [9]:
df.iloc[0,0]

7

In [69]:
df.values

array([[7,
        'الباعه المتجولون (ما عدا المواد الغذائيه) الذين لم يرد تصنيفهم – الاخرون'],
       [6, 'الزبال (جامع القمامه)'],
       [21,
        'المنتجات الخشبيه عامل يجمع ويركب الاثاث الخشبي والمواد المشابهه'],
       ...,
       [8, 'موظف مطبوعات / صحف حره'],
       [22, 'وقاد مراجل بخاريه (عطشجي) محرك بالبخار'],
       [22, 'وقاد مراجل بخاريه (عطشجي) محرك بالبخار']], dtype=object)

In [34]:
class DatasetInitializer(Dataset):
    """   """
    def __init__(self, csv_file, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.dataset_frame = pd.read_csv(csv_file)
        self.transform = transform


    def __len__(self):
        return len(self.dataset_frame)

    def __getitem__(self, idx):
        job_id = self.dataset_frame.iloc[idx, 0]
        job_name = self.dataset_frame.iloc[idx, 1]
        sample = {'job_id': job_id, 'job_name': job_name}
#         sample = [job_id,job_name]

        if self.transform:
            sample = self.transform(sample)

        return sample


In [35]:
dset = DatasetInitializer('../input/jobs_input.csv')

In [36]:
dset[9]

{'job_id': 7, 'job_name': 'بائع اقمشه'}

In [37]:
len(dset)

967

In [21]:
import pandas as pd
import torch
from torch.utils.data import Dataset
from sklearn.model_selection import train_test_split



class DatasetInitializer:
    def __init__(self, args):
        df = pd.read_csv(args['input_dir'] + args['train_csv'])
        df = df.sample(frac=1, random_state=args['random_seed'])
        test_frac = (len(df) * args['test_frac'])/(len(df) - (len(df)*args['valid_frac']))
        # generates from df an array
        self.train_x, self.valid_x = train_test_split(df.values     ,test_size=0.1  ,random_state=120)
        self.train_x, self.test_x  = train_test_split(self.train_x  ,test_size=0.1  ,random_state=120)

        print("Data Loaded Successfully!")
        print("train_size: {0} \t valid_size: {1} \t test_size: {2}".format(len(self.train_x), len(self.valid_x), len(self.test_x)))










class DatasetSplittingGenerator(Dataset):
    """   """
    def __init__(self, dataset_initialized ,args,split_part_name,transform=None):
        """
        Args:
            args: global experiment arguments
            split_part_name: which data portion to be loaded
            transform (callable, optional): Optional transform to be applied on a sample.
        """
        self.args = args
        self.split_part_name = split_part_name
        if self.split_part_name == 'train':
            self.x = dataset_initialized.train_x
        elif self.split_part_name == 'valid':
            self.x = dataset_initialized.valid_x
        elif self.split_part_name == 'test':
            self.x = dataset_initialized.test_x
        self.transform = transform


    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        job_id = self.x[idx, 0]
        full_job = self.x[idx, 1]
        pair = (job_id,full_job)

        if self.transform:
            sample = self.transform(pair)

        return pair


In [2]:
args = { "input_dir": "../input/",
         "train_csv": "jobs_input.csv",
        "random_seed": 120,
        "test_frac": 0.1, 
        "valid_frac": 0.1
       }

In [9]:
type(args['random_seed'])

int

In [22]:
jobs_full_data = DatasetInitializer (args)


Data Loaded Successfully!
train_size: 783 	 valid_size: 97 	 test_size: 87


In [23]:
jobs_full_data.train_x

array([[4, 'عامل المراسلات'],
       [24, 'عامل يشغل ماكينه تكرير السكر'],
       [25, 'عامل يشغل ماكينه الغسيل بالبخار'],
       ...,
       [24, 'عامل يشغل ماكينه تصنيع الفول المدمس'],
       [10, 'سائق مركبه يجرها حيوان (حنطور)'],
       [22, 'عامل تشغيل ماكينه تصنيع اللمبات الزجاجيه الفلورسنت']],
      dtype=object)

In [24]:
train_dataset = DatasetSplittingGenerator(jobs_full_data , args, 'train')

In [77]:
train_dataset[0][0]

4

In [ ]:
y_idx = transformation_helpers.string_to_int(train_dataset[0][1], self.add_sos, self.add_eos)
        return (pair[0], pair[1], y_idx)

In [26]:
len(train_dataset)

783

In [27]:
words = ['الباعه',
         'الزبال',
         'المنتجات',
         'بائع',
         'بحار',
         'بحري',
         'بواب',
         'تاجر',
         'تجهيز',
         'جامع',
         'جامعو',
         'جامعوا',
         'حارس',
         'حراث',
         'حصاد',
         'حفار',
         'حمال',
         'رئيس',
         'راعي',
         'سايس',
         'سائق',
         'سائقو',
         'شغال',
         'شيال',
         'صانع',
         'صناعه',
         'صياد',
         'طحن',
         'طواف',
         'عامل',
         'عزاق',
         'علاف',
         'عمال',
         'عماله',
         'عمل',
         'غواص',
         'فراش',
         'فنيو',
         'قارئ',
         'كاسح',
         'كبير',
         'كناس',
         'ماسح',
         'محصل',
         'مراقب',
         'مزارع',
         'مساعد',
         'مشغل',
         'معاون',
         'معدو',
         'مقلم',
         'ملاحظ',
         'منادي',
         'منسق',
         'منفذ',
         'موزع',
         'موظف',
         'وقاد',
         'المتجولون',
         'الخشبيه',
         '/',
         'ادوات',
         'اقمشه',
         'العاب',
         'بطاقات',
         'تذاكر',
         'جاز',
         'خطوط',
         'زهور',
         'سجاد',
         'شماسه',
         'صحف',
         'عطاره',
         'متجول',
         'مساحيق',
         'ملابس',
         'مناديل',
         'نظارات',
         'عادي',
         'ممتاز',
         'عجله',
         'خرده',
         'الخمور',
         'الدخان',
         'الشاي',
         'الفاكهه',
         'اللحوم',
         'الحطب',
         'المياه',
         'بيض',
         'مواد',
         'القمامه',
         'المهملات',
         'التذاكر',
         'سيارات',
         'يدوي',
         'خدم',
         'جراج',
         'تدريب',
         'ترام',
         'تروسيكل',
         'تريسيكل',
         'جرار',
         'حافله',
         'حيوانات',
         'دراجه',
         'سياره',
         'عربه',
         'قاطره',
         'لوري',
         'مترو',
         'مركبات',
         'مركبه',
         'المركبات',
         'امتعه',
         'السكر',
         'اسماك',
         'الاسماك',
         'الحبوب',
         'غابات',
         '/عامله',
         'اتصالات',
         'استبعاد',
         'اشاره',
         'اعداد',
         'المراسلات',
         'النظافه',
         'انتاج',
         'بيع',
         'تثبيت',
         'تجريف',
         'تجميع',
         'تحصيل',
         'تحويله',
         'تربيه',
         'ترزيم',
         'تركيب',
         'تزييت',
         'تسليك',
         'تسميد',
         'تشغيل',
         'تشهيلات',
         'تشييد',
         'تطريز',
         'تعبئه',
         'تفريخ',
         'تنظيف',
         'توزيع',
         'جلي',
         'جمع',
         'جني',
         'جني-',
         'حزم',
         'حمل',
         'خدمه',
         'خلطه',
         'رش',
         'زراعه',
         'زرعه',
         'سفلته',
         'شاطئ',
         'شحن',
         'شفط',
         'صيانه',
         'عجن',
         'غسيل',
         'غير',
         'فرز',
         'فرمله',
         'فرن',
         'فك',
         'في',
         'قطف',
         'كي',
         'مؤقت',
         'ماكينه',
         'مراعي',
         'مزرعه',
         'مصنع',
         'مقص',
         'مكافحه',
         'مناوله',
         'مهن',
         'هدم',
         'وزن',
         'يجمع',
         'يشغل',
         'التعبئه',
         'الشحن',
         'الكنس',
         'الهندسه',
         'ترتيب',
         'صيد',
         'الاطفال',
         'خيام',
         'مختبرات',
         'العدادات',
         'عداد',
         'الملاحين',
         'الشارع،',
         'احذيه',
         'جدران',
         'زجاج',
         'نقود',
         'ومسؤول',
         'حائز',
         'طباخ',
         'رافعه',
         'معدات',
         'مولدات',
         'الاطعمه',
         'اظافر',
         'صباغه',
         'مناره',
         'حركه',
         'خلطات',
         'علامات',
         'فرشات',
         'منشورات',
         'مطبوعات',
         'مراجل',
         'اطفال',
         'بموقف',
         'يانصيب',
         'وكروت',
         'وكليم',
         'ومجلات',
         'ورق',
         'القياده',
         'والمشروبات',
         'ومنتجاته',
         'والبن',
         'والخضراوات',
         'والاسماك',
         'والخشب',
         'السقا',
         'اعاده',
         'واعاده',
         'النفايات',
         'قياده',
         'بموتور',
         'زراعي',
         'لجر',
         'اتوبيس',
         'مربوطه',
         'ناريه',
         'اجره',
         'ملاكي',
         'ميكروباص',
         'نقل',
         'شاحنه',
         'فان',
         'اليه',
         'سكه',
         'تريك',
         'عام',
         'تدفع',
         'يجرها',
         'والالات',
         'شغاله',
         'بالشباك',
         'والتوابل',
         'لاسلكيه',
         'البضائع',
         'بالسكه',
         'طلاء',
         'المواد',
         'البطاقات',
         'تربه',
         'ونزح',
         'اجهزه',
         'افران',
         'الابواب',
         'الاقفال',
         'الغسالات',
         'حنفيات',
         'سخانات',
         'صناديق',
         'قواطه',
         'لوحات',
         'ماكينات',
         'محولات',
         'مدافئ',
         'مفاتيح',
         'مكونات',
         'منتجات',
         'وتركيب',
         'وحدات',
         'الخاماتتقشير',
         'قوالب',
         'ويشغل',
         'رسوم',
         'احياء',
         'اجزاء',
         'زراير',
         'وتجميع',
         'وتشحيم',
         'المجاري',
         'وري',
         'الات',
         'اله',
         'توربينات',
         'عربات',
         'غلايه',
         'كباس',
         'كراكه',
         'لودر',
         'محطه',
         'مصاعد',
         'مكبس',
         'هويس',
         'ومتابعه',
         'ومراقبه',
         'ونش',
         'واصلاحات',
         'المباني',
         'طرق',
         'الي',
         'وتغليف',
         'ابراج',
         'احواض',
         'الاثاث',
         'الالات',
         'النوافذ',
         'حظائر',
         'عبوات',
         'علي',
         'محطات',
         'مفروشات',
         'وصباغه',
         'وكي',
         'الجرائد',
         'المحاصيل',
         'بذور',
         'نباتات',
         'وتجهيز',
         '-',
         'وتسميه',
         'الفحم',
         'بالطريق',
         'حدائق',
         'خرسانيه',
         'الطرق',
         'مبيدات',
         'السدود',
         'الطفله',
         'الصحون',
         'ستائر',
         'وتنظيف',
         'ماهر',
         'الزجاجات',
         'وترتيب',
         'تحضير',
         'صهر',
         'وربط',
         'الغابات',
         'اماكن',
         'وتدريج',
         'مكوي',
         'باليد',
         'تسويه',
         'كشط',
         'طيور',
         'ماشيه',
         'مناول',
         'وتقليم',
         'حرائق',
         'فرديه',
         'وحزم',
         'ويركب',
         'المنشار',
         'وحده',
         'اليدويه',
         'والتفريغ',
         'ومن',
         'وموزعو',
         'الاخرون',
         'المنزليه',
         'المدنيه',
         'اواني',
         'الماشيه',
         'بالحدائق',
         'تعليميه',
         'الكهرباء',
         'مياه',
         'المطار،',
         'وارضيات',
         'عن',
         'متخصص',
         'بناء',
         'الحاويات',
         'ارضيه',
         'القوي',
         'الابقار',
         'اسفلت',
         'المرور',
         'اسفلتيه',
         'بخاريه',
         'عدا',
         'انابيب',
         'التنظيف',
         'السيارات',
         'كيروسين',
         'مقود',
         'البحر',
         'شرابات',
         'ولاعات',
         'دومنجي',
         'الروحيه',
         'والكاكاو',
         'والجوزيات',
         'التدوير',
         'تدويرها',
         'المنزل',
         'ذات',
         'ذو',
         'جذوع',
         'بالات',
         'موتوسيكل',
         'تاكسي',
         'اسعاف',
         'مغطي',
         'مكشوف',
         'او',
         'ثقيل',
         'حديد',
         'المناجم',
         'حيوان',
         'التي',
         'خادم',
         'البيتزا',
         'الهامبورجر',
         'ساندوتشات',
         'فول',
         'مشروبات',
         'مشويات',
         'التالفه',
         'الحديد',
         'الخزف',
         'الطائره',
         'القابله',
         'الارشاديه',
         'الثلوج',
         'التبريد',
         'تسخين',
         'غاز',
         'الجلديه',
         'المنشر',
         'كهربائيه',
         'صندوق',
         'ومستلزمات',
         'الغزل',
         'الورق',
         'الاسلحه',
         'مصابيح',
         '–',
         'قص',
         'مواقف',
         'مائيه',
         'الطائرات',
         'والبالوعات',
         'تسجيل',
         'التذريه',
         'الحرث',
         'عدد',
         'رفع',
         'لحمل',
         'مكوك',
         'الطوب',
         'تحميص',
         'حرق',
         'الهواء',
         'خلط',
         'ازاله',
         'التجفيف',
         'التغليف',
         'الكابلات',
         'اللصق',
         'تبييض',
         'تشذيب',
         'تشطيب',
         'تصنيع',
         'تعتيق',
         'تنعيم',
         'حبكه',
         'حفر',
         'حياكه',
         'خط',
         'خياطه',
         'دباغه',
         'درزه',
         'دق',
         'دك',
         'رصف',
         'شكه',
         'صقل',
         'ضغط',
         'عراوي',
         'فتح',
         'فرش',
         'فقاقيع',
         'كاسحه',
         'كبس',
         'كمش',
         'لف',
         'لقطه',
         'معالجه',
         'نقش',
         'هراس',
         'حراريه',
         'الرش',
         'الري',
         'تخريم',
         'الغلايات',
         'برجي',
         'ثابت',
         'حصيره',
         'ذي',
         'طبليه',
         'قفص',
         'كوبري',
         'متحرك',
         'محمل',
         'للمنتجات',
         'السباحه',
         'سمك',
         'المنجد',
         'وموكيت',
         'الجاف',
         'وسائل',
         'وستائر',
         'السجاد',
         'والمجلات',
         'مطبخ',
         'بريه',
         'المنتج',
         'العام',
         'وبساتين',
         'زينه',
         'اشجار',
         'تحميل',
         'العادمه',
         'والاواني',
         'ومفروشات',
         'وتلميع',
         'الكهنه',
         'الزجاج',
         'السفن',
         'ازهار',
         'اسطوانه',
         'باسقاط',
         'الارض',
         'نحل',
         'للمواد',
         'اطارات',
         'التوربينات',
         'الدراجات',
         'الدعامات',
         'الستائر',
         'العربات',
         'الكهروميكانيكيه',
         'المحركات',
         'المعدات',
         'المكونات',
         'الهاتف',
         'زجاجه',
         'لعب',
         'محركات',
         'وتشغيل',
         'الحزامي',
         'بلوره',
         'تشكيل',
         'البلوره',
         'التبخير',
         'التخريم',
         'التسميك',
         'التسويه',
         'الجبن',
         'الحقائب',
         'الحلوي',
         'السروح',
         'الغسيل',
         'المشروبات',
         'المكرونه',
         'النشر',
         'برش',
         'برم',
         'بستره',
         'تتبيل',
         'تجفيف',
         'تجميد',
         'تحميد',
         'تخليل',
         'تخمير',
         'تزرير',
         'تصفيه',
         'تطبيع',
         'تعقيم',
         'تعليب',
         'تفصيل',
         'تقشير',
         'تقطير',
         'تقطيع',
         'تكرير',
         'تمليح',
         'تنقيه',
         'حقائب',
         'سحب',
         'شد',
         'صنع',
         'طبخ',
         'عصاره',
         'عصر',
         'عصير',
         'غسل',
         'فتق',
         'فرم',
         'فريزا',
         'قطع',
         'كلندره',
         'لب',
         'مزج',
         'مناخل',
         'نقر',
         'هدرجه',
         'هضم',
         'مجففات',
         'الذين',
         'اليدوي',
         'اليهم',
         'الطرود',
         'الفندق',
         'المكاتب',
         'المكتب',
         'البذار',
         'التسميد',
         'الحصاد',
         'تجليد',
         'تخريس',
         'تذهيب',
         'تصفيح',
         'تغليف',
         'سلفنه',
         'طباعه',
         'طي',
         'الحقليه',
         'والاحياء',
         'الملابس',
         'والنفايات',
         'والبساتين',
         'التشييد',
         'الزراعه',
         'و',
         'البحاره',
         'المحطه،',
         'وسلالم',
         'البيع',
         'ارشاد',
         'الوجبات',
         'اعلانات',
         'عطشجي',
         'ما',
         'منزليه',
         'ومواد',
         'بوتاجاز',
         'بمبوطي',
         'الاخري',
         'بدال',
         'مقطوره',
         'الاشجار',
         'لاداء',
         'قلاب',
         'والمحاجر',
         'بمقطوره',
         'حنطور',
         'تجرها',
         'البوص',
         'وطعميه',
         'بارده',
         'ساخنه',
         'دجاج',
         'السكك',
         'للتدوير',
         'والعلامات',
         'التروس',
         'والنسيج',
         'الخام',
         'والذخيره',
         'فلوريه',
         'وتحريز',
         'محولجي',
         'الارفف',
         'معدنيه',
         'والمعدات',
         'بيانات',
         'والجني',
         'ميكانيكيه',
         'الاثقال',
         'رولات',
         'الاخشاب',
         'بالمناجم',
         'السفينه',
         'والبلاط',
         'ضاغط',
         'جراف',
         'وخلط',
         'الشعر',
         'صمغ',
         'بين',
         'ووضع',
         'الديكاتور',
         'رقائق',
         'المنسوجات',
         'اللون',
         'ازرار',
         'اوفر',
         'اللمبات',
         'اتوماتيكيه',
         'صناعيه',
         'سوائل',
         'مساحي',
         'وغسيل',
         'انفاق',
         'خنادق',
         'سطحي',
         'الفراء',
         'القبعات',
         'المفروشات',
         'الاسفلت',
         'الخرسانه',
         'دائري',
         'متعدده',
         'الجلود',
         'الخوازيق',
         'وتسويه',
         'وتوزيع',
         'وتحميل',
         'الالياف',
         'الياف',
         'خيوط',
         'الملفات',
         'الاقمشه',
         'وتعبئه',
         'وغلق',
         'الالومنيوم',
         'تحضيرات',
         'الحرير',
         'الطريق',
         'معادن',
         'زجاجيه',
         'لبناء',
         'برج',
         'بكره',
         'صاري',
         'واغلاق',
         'النقل',
         'بالجمله',
         'للتخزين',
         'الخضر',
         'للمنازل',
         'الحدائق',
         'وشجيرات',
         'سفن',
         'شاحنات',
         'جوي',
         'ومنتجات',
         'رباطه',
         'العمله',
         'محدده',
         'النظارات',
         'المعدني',
         'الزراعيه',
         'الصناعيه',
         'الميكانيكيه',
         'الانشائيه',
         'البحريه',
         'البخاريه',
         'الالكترونيه',
         'السمعيه',
         'الكهربائيه',
         'المعدنيه',
         'الترموس',
         'الدوائر',
         'التعدين',
         'الطباعه',
         'المشغولات',
         'الاحتراق',
         'تحريك',
         'الانتاج',
         'المظلات',
         'البلاستيك',
         'المطاط',
         'وتقطيع',
         'شرائط',
         'تدوير',
         'دبغ',
         'جرش',
         'لصناعه',
         'بعد',
         'فاره',
         'رابوه',
         'بانواعه',
         'بانواعها',
         'بالبخار',
         'منشره',
         'الخبز',
         'الدقيق',
         'الشكولاته',
         'علف',
         'التبغ',
         'الالبان',
         'عجينه',
         'الارز',
         'الخضراوات',
         'محلول',
         'حبوب',
         'الخضروات',
         'الخشب',
         'اعواد',
         'البوظه',
         'الحمص',
         'الطحينه',
         'الفول',
         'القهوه',
         'سواك',
         'اللفائف',
         'نعال',
         'وجرش',
         'وعصير',
         'وتخمير',
         'جاف',
         'صنفره',
         'وتصفيه',
         'السفر',
         'البن',
         'نعل',
         'وجه',
         'كعب',
         'الاحذيه',
         'البسكويت',
         'البيره',
         'التورتات',
         'الحلاوه',
         'الخيام',
         'السيجار',
         'الكحوليات',
         'اللبان',
         'اللبن',
         'المربات',
         'النبيذ',
         'النماذج',
         'خميره',
         'زيوت',
         'وتقطير',
         'الخضراوات/',
         'وتنقيه',
         'التوابل',
         'الطماطم',
         'خشب',
         'نماذج',
         'القشره',
         'اوراق',
         'الملبوسات',
         'الزيوت',
         'استخلاص',
         'صهاريج',
         'لم',
         'وشيالو',
         'والفنادق',
         'وتغريه',
         'بالسلك',
         'الاستمارات',
         'ملازم',
         'اطباق',
         'دبوس',
         'وتخريز',
         'الكتب',
         'اوفست',
         'بارزه',
         'غائره',
         'ملونه',
         'منسوجات',
         'شريطي',
         'محدود',
         'المائيه',
         'الطيور',
         'وتربيه',
         'والبناء',
         'المختلطه',
         'الحديقه...',
         'واسطح',
         'الناس',
         'اللب',
         'المطار',
         'تعمل',
         'السريعه',
         'حره',
         'محرك',
         'الغذائيه',
         'قفازات',
         'شباشب',
         'الاعمال',
         'مغطاه',
         'لمسافات',
         'الحيوانات',
         'خادمه',
         'والكرتون',
         'والماكينات',
         'حصر',
         'شوكه',
         'ونقلها',
         'والسيراميك',
         'والصيني',
         'حلاقه',
         'مراحل',
         'العلامات',
         'الحبال',
         'السليكون',
         'الحراريه',
         'وسجاد',
         'الزجاجيه',
         'الجينز',
         'حفاره',
         'الجاهزه',
         'التليفزيون',
         'الراديو',
         'الابر',
         'بالخرسانه',
         'الاتربه',
         'ياقه',
         'النسيجيه',
         'النسيج',
         'مزلقان',
         'بالاسفلت',
         'مقصدار',
         'الخياطه',
         'لزياده',
         'ضد',
         'جريدر',
         'المساكن',
         'وسيور',
         'ارصفه',
         'الرجل',
         'البالات',
         'البساتين',
         'ناقلات',
         'خضروات',
         'الشوارع',
         'المطابخ',
         'للمركبات',
         'القصب',
         'مخلفات',
         'رول',
         'الشاحنات',
         'الدقيقه',
         'من',
         'المطبوعه',
         'المعادن',
         'الداخلي',
         'التربه',
         'المجوهرات',
         'اقلام',
         'السمندر',
         'الشريطي',
         'الزينه',
         'البذور',
         'خلاصه',
         'المغسله',
         'والاحزمه',
         'الاطواق',
         'المسكره',
         'الشعريه',
         'الادمه',
         'وتمليحها',
         'بجميع',
         'تفتيح',
         'الدباغه',
         'غلي',
         'الثقاب',
         'بالطحينه',
         'المدمس',
         'سريعه',
         'يدويه',
         'الاسنان',
         'الحميد',
         'وغربله',
         'السمسم',
         'العنب',
         'الاخضر',
         'الحذاء',
         'والسكاكر',
         'الكحوليه',
         'والكعك',
         'الطحينيه',
         'روح',
         'المركز',
         'الباترونات',
         'الجوزيات',
         'الطحين',
         'المخابز',
         'لحوم',
         'الخل',
         'الحبيبي',
         'شق',
         'طحين',
         'والدهون',
         'يعمل',
         'العصائر',
         'تطهير',
         'يرد',
         'الامتعه',
         'وغيرها',
         'والتخزين',
         'الغرس',
         'اللولبي',
         'ليثوغرافي',
         'تيبو',
         'بالرول',
         'الفلسكو',
         'متصله',
         'شيتر',
         'المهاره',
         'محدودي',
         'والمواشي',
         'مخبريه',
         'للمخارج',
         'البقوليات',
         'بالمراجل',
         'الخشبي',
         'اعلام',
         'خمور',
         'المختلفه',
         'طويله',
         'دولي',
         'الاستخدام',
         'يشارك',
         'البهارات',
         'الثابته',
         'والفخار',
         'الطبيعي',
         'الصباغه',
         'الرام',
         'الفلورسنت',
         'تاسيس',
         'بالسيارات',
         'ورش',
         'الجليد',
         'وزنه',
         'تسرب',
         'عائمه',
         'سكك',
         'القدم',
         'والمشاتل',
         'والطائرات',
         'الرصاص',
         'لقطع',
         'والورق',
         'والتلحيم',
         'وطبخها',
         'المراحل',
         'الشغير',
         'بوظه',
         'الذوبان',
         'نكاشات',
         'وما',
         'وغير',
         'والفطائر/',
         'الخمر',
         'علكه',
         'مثل',
         'الخنزير',
         'المولت',
         'الطعام',
         'غسلها',
         'تسحيب',
         'الابلكاش',
         'تحت',
         'السكريه',
         'تصنيفهم',
         'بالطبق',
         'وصيانه',
         'والمواد',
         'السجائر',
         'محلي',
         'والنكهات',
         'بالمصنع',
         'للنسيج',
         'الطبقه',
         'حديديه',
         'للحواسب',
         'الاطقم',
         'ترطيب',
         'عربيه',
         'حفظ',
         'الكنافه',
         'النشا',
         'المعجنات',
         'قصب',
         'اشراف',
         'السائل',
         'مساعدون',
         'الطوارئ',
         'المشابهه',
         'وتجفيف',
         'سلق،',
         'وفصل',
         'والتي',
         'DVD',
         'للطرق',
         'والمحركات',
         'الخ',
         'توليف',
         'فاكهه',
         'القطايف',
         'والمكسرات',
         'وترشح',
         'لانتاج',
         'يثبت',
         'تدخل',
         'الممهده',
         'القمح',
         'البسطرمه',
         'النخيل',
         'عناصر',
         'مدخله',
         'الكاكاو',
         'والنقانق',
         'والوجبات',
         '...',
         'اي',
         'بحسب',
         'التعليمات',
         'ويطبق',
         'تعليمات',
         'السلامه',
         'والصحه',
         'المهنيه',
         'اخري',
         'سائله',
         'صلبه',
         'نزع',
         'سيقان',
         'توليفه',
         'وعلف',
         'الحيوان',
         'لوجبات', ]
# pad must be index 0
vocab_list = ['<pad>', '<sos>', '<unk>', '<eos>'] + words
vocab = dict((c, i) for i, c in enumerate(vocab_list))
inv_vocab = dict((i, c) for i, c in enumerate(vocab_list))

In [30]:
vocab['<pad>']

0

In [28]:
import random as rn
from torch import nn
import numpy as np
from numpy.random.mtrand import rand, randint as random_randint, choice as random_choice



In [ ]:
char_embedding = nn.Embedding(len(vocab_list), 100 , padding_idx=cls.vocab['<pad>'])
if args.fixed_embeddings:
    char_embedding.weight.requires_grad = False

In [61]:
# rep = [vocab.get('بائع ادوات كهربائية', vocab['<unk>'])]
rep = list(map(lambda x: vocab.get(x, vocab['<unk>']), str('بائع ادوات كهربائيه').split()))
 

In [60]:
list(map(lambda x: x, str('بائع ادوات كهربائية').split()))

['بائع', 'ادوات', 'كهربائية']

In [62]:
rep

[7, 65, 452]

In [34]:
vocab.get('عامل', vocab['<unk>'])

2

In [36]:
vocab.get('عامل')

33

In [63]:
rep = [vocab['<sos>']] + rep 
rep = rep + [vocab['<eos>']] 

In [83]:
len(rep)

5

In [72]:
sos_index = rep.index(vocab['<sos>'])
eos_index = rep.index(vocab['<eos>'])
eos_index , sos_index

(4, 0)

In [73]:
sentence = [inv_vocab.get(i, ".") for i in rep[sos_index+1 :eos_index]]
sentence

['بائع', 'ادوات', 'كهربائيه']

In [74]:
nn.Embedding(len(vocab_list), 100, padding_idx = vocab['<pad>'])

Embedding(1150, 100, padding_idx=0)

In [81]:
(train_dataset[0][0], train_dataset[0][1], rep)

(4, 'عامل المراسلات', [1, 7, 65, 452, 3])

In [79]:
rep

[1, 7, 65, 452, 3]

In [82]:
from torch.utils.data import DataLoader


In [ ]:
DataLoader(test_dataset,  batch_size=args.batch_size, shuffle=False, num_workers=args.num_workers, collate_fn=train_dataset.collate_fn, **kwargs)
